<a href="https://colab.research.google.com/github/yenlung/Python-AI-Book/blob/main/%E5%86%92%E9%9A%AA%E5%89%AF%E6%9C%AC06_%E7%94%A8OpenAI_API%E6%89%93%E9%80%A0%E8%87%AA%E5%B7%B1%E7%9A%84ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這裡是因為 Colab 改版, 目前才需要這麼複雜的安裝。

In [1]:
!pip install -U tensorflow-probability
!pip install typing-extensions --upgrade
!pip install gradio
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.22.0
    Uninstalling tensorflow-probability-0.22.0:
      Successfully uninstalled tensorflow-probability-0.22.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 使用 OpenAI 的 API

我們先來安裝 `openai` 套件, 還有快速打造 Web App 的 `gradio`。

In [3]:
# 本來在這裡裝 openai 和 gradio, 但前面已經裝了
#
#!pip install openai
#!pip install gradio

### 2. 申請自己的 OpenAI API 金鑰

請先申請自己的 OpenAI API 金鑰

[`https://platform.openai.com`](https://platform.openai.com)




### 3. 使用 ChatGPT API

首先使用 `openai` 套件。

In [4]:
from openai import OpenAI

把自己的金鑰貼上。

In [5]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="你的 API 金鑰",
)

ChatGPT API 的重點是要把之前對話的內容送給 ChatGPT, 然後他就會有個適當的回應!

角色 (`role`) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

In [6]:
prompt = "我今天心情很不好。"

messages = [{"role":"system", "content":"你是一個非常溫暖的對話機器人，回應都簡短，儘量不要超過二十個字，而且有同理心。"},
            {"role": "user", "content": prompt}]

In [7]:
messages

[{'role': 'system', 'content': '你是一個非常溫暖的對話機器人，回應都簡短，儘量不要超過二十個字，而且有同理心。'},
 {'role': 'user', 'content': '我今天心情很不好。'}]

現在我們來接 ChatGPT 的回應。

In [8]:
chat_completion = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
)

In [9]:
chat_completion

ChatCompletion(id='chatcmpl-8XT4spSlQ6esL4VNGZQlbmEy39i50', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='怎麼了？可以跟我分享嗎？', role='assistant', function_call=None, tool_calls=None))], created=1702987290, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=77, total_tokens=93))

In [10]:
chat_completion.choices[0].message.content

'怎麼了？可以跟我分享嗎？'

In [11]:
reply = chat_completion.choices[0].message.content

In [12]:
reply

'怎麼了？可以跟我分享嗎？'

In [13]:
messages.append({"role":"assistant", "content":reply})

In [14]:
messages

[{'role': 'system', 'content': '你是一個非常溫暖的對話機器人，回應都簡短，儘量不要超過二十個字，而且有同理心。'},
 {'role': 'user', 'content': '我今天心情很不好。'},
 {'role': 'assistant', 'content': '怎麼了？可以跟我分享嗎？'}]

### 4. 打造一個可以一直說下去的 ChatGPT

In [15]:
icon = "(*ﾟ▽ﾟ*): "
messages = [{"role":"system", "content":"你是一個非常溫暖的對話機器人，回應都簡短，儘量不要超過二十個字，而且有同理心。"}]

while True:
    prompt = input('> ')
    if 'bye' in prompt:
        print('再見, 下次再聊!')
        break
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        )

    reply = chat_completion.choices[0].message.content
    print(icon + reply)
    print()
    messages.append({"role": "assistant", "content": reply})

> 我覺得很難過
(*ﾟ▽ﾟ*): 為什麼呢？可以跟我說說嗎？

> Python 都不會寫
(*ﾟ▽ﾟ*): 沒關係，學習一門程式語言需要時間和練習。你可以從基礎開始，慢慢學習，我相信你能掌握Python的！加油！

> 同學都排擠我
(*ﾟ▽ﾟ*): 這讓你感到不舒服。試著與他們溝通，或尋找其他人與你有共同興趣的朋友。你不是孤單的，我在這裡支持你。

> 先這樣, bye
再見, 下次再聊!


### 5. 最後用 `gradio` 打造 ChatGPT 網路應用程式!

In [16]:
import gradio as gr

In [17]:
messages = [{"role":"system", "content":"你是一個非常溫暖的對話機器人，回應都簡短，儘量不要超過二十個字，而且有同理心。"}]


In [18]:
def pipi(prompt, history):
    history = history or []
    global messages

    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    history = history + [[prompt, reply]]

    return history, history

In [19]:
chatbot = gr.Chatbot()

In [22]:
iface = gr.Interface(pipi,
                     inputs=["text", "state"],
                     outputs=[chatbot, "state"],
                     title="療癒系拍拍機器人",
                     description="你有什麼心事, 歡迎來找我聊聊 :)")

In [23]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a070a50d7289189f1f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
